#  XAI – Inteligencia Artificial Explicable

En esta sección se aplican técnicas de **Inteligencia Artificial Explicable (XAI)** con el objetivo de entender mejor cómo y por qué los modelos realizan sus predicciones. Esto resulta especialmente útil para validar la coherencia del modelo y detectar posibles sesgos o patrones inesperados.

---

## Objetivo

El propósito es analizar las decisiones del modelo entrenado (Gradient Boosting optimizado con Optuna de GS + M1000) utilizando dos herramientas complementarias:

- **LIME (Local Interpretable Model-Agnostic Explanations):**  
  Explica la predicción de un partido individual mostrando cómo cada variable influyó en la decisión.

- **SHAP (SHapley Additive exPlanations):**  
  Proporciona interpretaciones tanto locales (una predicción concreta) como globales (importancia de variables a nivel general).

---

## Herramientas y técnicas

| Técnica | Tipo de explicación | Aplicación en el proyecto              |
|---------|---------------------|-----------------------------------------|
| **LIME** | Local               | Explicación detallada de un partido específico: ¿por qué el modelo predijo que ganaba el favorito? |
| **SHAP** | Local & Global      | - Análisis de un partido concreto.<br>- Análisis general de las variables más importantes en todas las predicciones. |

---

## Procedimiento

1. **Seleccionar un partido de ejemplo** del dataset de Grand Slams + Masters 1000 para explicar la predicción.
2. **Aplicar LIME**:
   - Generar una explicación que indique qué características empujaron la predicción hacia “ganar” o “no ganar”.
3. **Aplicar SHAP**:
   - Explicar cómo cada variable influyó en la predicción de un partido específico.
   - Visualizar la importancia global de las variables en el comportamiento general del modelo.
4. **Extraer conclusiones** sobre la coherencia del modelo y la relevancia de las variables.

---

## Resultados esperados

- Gráficas de LIME mostrando la contribución de cada variable en un partido concreto.
- Gráficas de SHAP para:
  - Explicación local (partido individual).
  - Explicación global (importancia de variables en el dataset completo).

---

## Justificación

Aplicar estas técnicas permitirá responder preguntas como:
- ¿El modelo toma decisiones razonables y basadas en datos relevantes?
- ¿Existen variables con demasiada influencia que puedan generar sesgos?
- ¿Es consistente la importancia de las variables entre los partidos y en el conjunto global?

In [ ]:
# Cargar datos 
ruta_base = r"./Modelo_Completo/columnas_añadidas"
nombre_archivo = "escaladofinal.csv"
ruta_completa = os.path.join(ruta_base, nombre_archivo)

df = pd.read_csv(ruta_completa, delimiter=";", decimal=",")
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
df = df.sort_values(by="Date")

# Selección de características y target
features = [
    "Surface_WinRate_Favorite", "Surface_WinRate_Not_Favorite",
    "Surface_Matches_Favorite", "Surface_Matches_Not_Favorite",
    "WinStreak_Favorite", "WinStreak_Not_Favorite", "Win_Streak_Diff",
    "Rank_Favorite", "Rank_Not_Favorite", "Rank_Diff_Signed", "Rank_Diff_Abs",
    "Masters1000_Favorite", "Masters1000_Not_Favorite",
    "GrandSlams_Favorite", "GrandSlams_Not_Favorite"
]
target = "Favorite_Wins"

X = df[features]
y = df[target]

In [ ]:


# --- Modelo (Optuna) ---
model = GradientBoostingClassifier(
    learning_rate=0.014538386775171604,
    n_estimators=475,
    max_depth=4,
    subsample=0.5162888716243986,
    random_state=42
)


model.fit(X, y)


# Seleccionar 2 partidos extremos:
#   - HIGH: P(Gana favorito) >= 0.70
#   - LOW : P(Gana favorito) <= 0.30


# Probabilidades por clase según el modelo
probs = model.predict_proba(X)  # shape (n_samples, n_clases)

# IDENTIFICAR el índice de la clase "Gana favorito".
classes = model.classes_
name_map = {0: "No gana favorito", 1: "Gana favorito"}
# Fallback por si las clases no son 0/1:
class_names_by_model = [name_map.get(c, str(c)) for c in classes]

# Índice de la clase "Gana favorito" (si no existe 1, toma la de mayor etiqueta como positivo)
if 1 in classes:
    pos_idx = int(np.where(classes == 1)[0][0])
else:
    pos_idx = int(np.argmax(classes))  #la clase "mayor" como positiva

p_fav = probs[:, pos_idx]  # P(Gana favorito)

# Índices que cumplen los umbrales
high_candidates = np.where(p_fav >= 0.70)[0]
low_candidates  = np.where(p_fav <= 0.30)[0]

rng = np.random.default_rng(42)

def pick_fallback(candidates, target_prob, pick_low=True):
    """Si no hay candidatos, elige el más cercano al target_prob."""
    if len(candidates) > 0:
        return int(rng.choice(candidates))
    # Fallback: índice cuya probabilidad esté más cerca de target_prob
    return int(np.argmin(np.abs(p_fav - target_prob)))

idx_high = pick_fallback(high_candidates, 0.85, pick_low=False)
idx_low  = pick_fallback(low_candidates, 0.15, pick_low=True)

print(f"Partido HIGH (fav >= 0.70) -> idx {idx_high}, P_fav={p_fav[idx_high]:.3f}")
print(f"Partido LOW  (fav <= 0.30) -> idx {idx_low},  P_fav={p_fav[idx_low]:.3f}")

# -------------------------------
# LIME: configuramos el explicador
# -------------------------------
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X),
    feature_names=list(X.columns) if 'features' not in globals() else features,
    class_names=class_names_by_model,   # importante: MISMO ORDEN que model.predict_proba
    mode="classification",
    discretize_continuous=True,        
    random_state=42
)

# Generar explicaciones para ambos casos
num_features = 10  # top-10 variables

# Caso HIGH (favorito muy probable)
exp_high = explainer.explain_instance(
    data_row=X.iloc[idx_high].values,
    predict_fn=model.predict_proba,
    num_features=num_features
)
exp_high.show_in_notebook(show_table=True, show_all=False)
exp_high.save_to_file('lime_explanation_high_70.html')
print("Explicación HIGH guardada en lime_explanation_high_70.html")

# Caso LOW (favorito poco probable)
exp_low = explainer.explain_instance(
    data_row=X.iloc[idx_low].values,
    predict_fn=model.predict_proba,
    num_features=num_features
)
exp_low.show_in_notebook(show_table=True, show_all=False)
exp_low.save_to_file('lime_explanation_low_30.html')
print("Explicación LOW guardada en lime_explanation_low_30.html")


> Importante: En las gráficas de LIME los bloques azules y naranjas **no representan probabilidades absolutas** que se puedan sumar (por ejemplo, no se espera que los azules sumen exactamente 0.25).  
> Lo que muestran son **contribuciones locales**:  
> - **Naranja** → variables que empujan la predicción hacia *“Gana favorito”*.  
> - **Azul** → variables que empujan la predicción hacia *“No gana favorito”*.  
> - **La longitud del bloque** refleja la **fuerza relativa** de esa variable en este partido.  
> El balance global de todas las contribuciones aproxima la probabilidad final (ej. 75% favorito – 25% no favorito), pero no es una suma exacta.


### Caso 1 – Favorito (75% probabilidad de ganar)

- El modelo predice que el **favorito ganará con un 75% de probabilidad**.  
- Factores que favorecen al favorito:  
  - **Mal rendimiento del no favorito en esta superficie** (`Surface_WinRate_Not_Favorite = 0.70`).  
  - **Mejor racha de victorias del favorito** (`Win_Streak_Diff = 0.30`).  
- Factores que van en contra del favorito:  
  - **Pocos partidos del favorito en esta superficie** (`Surface_Matches_Favorite = -0.59`).  
  - **Cierta experiencia del no favorito en Masters1000** (`Masters1000_Not_Favorite = -0.14`).  

Aunque existen debilidades (poca experiencia del favorito en la superficie), el mal rendimiento histórico del no favorito pesa más, lo que inclina la predicción hacia el favorito.  


### Caso 2 – No favorito (72% probabilidad de ganar)

El modelo predice que el **no favorito ganará con un 72% de probabilidad**, frente a un 28% del favorito.

**Factores que favorecen al no favorito:**
- Buen rendimiento en la superficie (`Surface_WinRate_Not_Favorite = 1.12`).
- Mayor número de partidos jugados en la superficie (`Surface_Matches_Not_Favorite = 1.01`).
- Diferencia de ranking a su favor (`Rank_Diff_Signed = 0.36`).

**Factores que van en contra del no favorito (a favor del favorito):**
- Buen winrate del favorito en la superficie (`Surface_WinRate_Favorite = -2.67`).
- Experiencia del favorito en esta superficie (`Surface_Matches_Favorite = -0.74`).
- Experiencia del no favorito en torneos Masters1000 (`Masters1000_Not_Favorite = -0.14`).

Aunque el favorito tiene un buen historial en la superficie y experiencia en torneos importantes, el modelo considera más relevantes el **rendimiento reciente y la experiencia acumulada del no favorito en la superficie**, lo que inclina la predicción a su favor con un 72% de probabilidad.


### SHAP (SHapley Additive exPlanations)

Tras aplicar LIME para entender casos individuales, utilizamos **SHAP** como técnica complementaria de Inteligencia Artificial Explicable.  
SHAP se basa en los valores de *Shapley* de la teoría de juegos y permite descomponer la predicción de un modelo en **contribuciones de cada variable**.

- **Explicación local:** muestra cómo cada característica concreta ha influido en la predicción de un partido específico.  
- **Explicación global:** permite identificar qué variables son más relevantes en el comportamiento general del modelo y cómo afectan sus valores (altos o bajos) a la probabilidad de victoria.

De esta forma, SHAP nos ayuda a validar si el modelo toma decisiones coherentes y a detectar posibles sesgos en el conjunto de datos.


In [ ]:

# Definir y entrenar modelo

# Modelo final optimizado con Optuna
model = GradientBoostingClassifier(
    learning_rate=0.014538386775171604,
    n_estimators=475,
    max_depth=4,
    subsample=0.5162888716243986,
    random_state=42
)

# Entrenar con todo el dataset
model.fit(X, y)

# SHAP: explicaciones

shap.initjs()  # habilita visualizaciones interactivas en notebook

# Crear explicador basado en árboles
explainer = shap.TreeExplainer(model)
raw_shap_values = explainer.shap_values(X)

# Índice de la clase "Gana favorito"
classes = getattr(model, "classes_", np.array([0, 1]))
pos_idx = int(np.where(classes == 1)[0][0]) if 1 in classes else int(np.argmax(classes))

# Normalizar salida para binario/multiclase
if isinstance(raw_shap_values, list):
    sv = raw_shap_values[pos_idx]
    expected = explainer.expected_value[pos_idx]
else:
    sv = raw_shap_values
    expected = explainer.expected_value


# Explicación global

# importancia de variables (ranking)
plt.figure()
shap.summary_plot(sv, X, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig("shap_global_importance_bar.png", dpi=160, bbox_inches="tight")
plt.show()

# b) Dispersión (beeswarm): muestra cómo afectan valores altos/bajos
plt.figure()
shap.summary_plot(sv, X, show=False)
plt.tight_layout()
plt.savefig("shap_global_beeswarm.png", dpi=160, bbox_inches="tight")
plt.show()

print("Guardadas: shap_global_importance_bar.png, shap_global_beeswarm.png")


# Explicaciones locales

# Elegimos partidos extremos como en LIME
probs = model.predict_proba(X)[:, pos_idx]
high_candidates = np.where(probs >= 0.70)[0]
low_candidates  = np.where(probs <= 0.30)[0]

rng = np.random.default_rng(42)
def pick(cands, target):
    return int(rng.choice(cands)) if len(cands) else int(np.argmin(np.abs(probs - target)))

idx_high = pick(high_candidates, 0.85)
idx_low  = pick(low_candidates, 0.15)

print(f"Partido HIGH (favorito fuerte): idx={idx_high}, P_fav={probs[idx_high]:.2f}")
print(f"Partido LOW  (no favorito fuerte): idx={idx_low}, P_fav={probs[idx_low]:.2f}")

# a) Force plot (HIGH)
fig = shap.force_plot(expected, sv[idx_high, :], X.iloc[idx_high, :], matplotlib=True, show=False)
plt.title(f"SHAP force plot – Favorito ≈ {probs[idx_high]:.2f}")
plt.savefig("shap_local_force_high.png", dpi=160, bbox_inches="tight")
plt.show()

# b) Force plot (LOW)
fig = shap.force_plot(expected, sv[idx_low, :], X.iloc[idx_low, :], matplotlib=True, show=False)
plt.title(f"SHAP force plot – Favorito ≈ {probs[idx_low]:.2f}")
plt.savefig("shap_local_force_low.png", dpi=160, bbox_inches="tight")
plt.show()

print("Guardadas: shap_local_force_high.png, shap_local_force_low.png")



### Explicaciones con SHAP

SHAP (SHapley Additive exPlanations) permite descomponer las predicciones del modelo en **contribuciones de cada variable**.  
Los valores SHAP se interpretan así:  
- **Positivos (rojo)** → aumentan la probabilidad de que gane el favorito.  
- **Negativos (azul)** → reducen la probabilidad de que gane el favorito.  



---

#### 1. Importancia global (bar plot)

El gráfico de barras muestra la **importancia media absoluta** de cada variable en todas las predicciones.  
Las más influyentes en el modelo son:

- **Surface_WinRate_Favorite**  
- **Rank_Favorite**  
- **Surface_WinRate_Not_Favorite**  
- **Rank_Diff_Signed** y **Rank_Diff_Abs**

Esto confirma que el modelo se apoya sobre todo en **rendimiento en superficie** y **ranking de los jugadores** para tomar decisiones.

---

#### 2. Comportamiento global (beeswarm)

El gráfico de dispersión (beeswarm) muestra, además de la importancia, **cómo influyen los valores altos o bajos** de cada variable:

- **Surface_WinRate_Favorite** alto (rojo) → aumenta la probabilidad de victoria del favorito.  
- **Surface_WinRate_Not_Favorite** alto (rojo) → reduce la probabilidad de victoria del favorito.  
- **Rank_Favorite** bajo (mejor ranking) → favorece al favorito; alto (peor ranking) → lo perjudica.

Esto refleja patrones consistentes y lógicos: el modelo confía en la calidad del favorito y en el mal desempeño del no favorito en la superficie.

---

#### 3. Explicación local – Caso LOW (P_fav ≈ 0.28)

El modelo predice que el favorito tiene solo un **28% de probabilidad de ganar**.  

Principales factores en contra del favorito:
- **Surface_WinRate_Favorite** (−0.51) → el favorito tiene mal rendimiento en esa superficie.  
- **Surface_WinRate_Not_Favorite** (−0.39) → el no favorito tiene buen winrate en la superficie.  
- **Rank_Diff_Abs** (−0.23) y **Rank_Diff_Signed** (−0.22) → la diferencia de ranking no favorece al favorito.  
- **Surface_Matches_Not_Favorite** (−0.16) → el no favorito ha jugado más en la superficie.  

El no favorito aparece mejor posicionado por experiencia en la superficie y ranking, lo que lleva al modelo a asignarle ventaja.

---

#### 4. Explicación local – Caso HIGH (P_fav ≈ 0.75)

En este partido, el favorito tiene un **75% de probabilidad de ganar**.  

Principales factores a favor del favorito:
- **Surface_Matches_Not_Favorite** (+0.15) → poca experiencia del no favorito en la superficie.  
- **Rank_Diff_Abs** (+0.14) y **Rank_Diff_Signed** (+0.13) → la diferencia de ranking favorece al favorito.  
- Aportes menores: **Surface_WinRate_Favorite** (+0.04), **Rank_Not_Favorite** (+0.04), **Win_Streak_Diff** (+0.03).  

Factores en contra:
- **Surface_WinRate_Not_Favorite** (−0.14).  
- **Rank_Favorite** (−0.09).  
- **Masters1000_Favorite** (−0.04).  

 Aunque existen variables que restan (por ejemplo, cierto rendimiento del no favorito), la ventaja en **ranking** y la **poca experiencia del no favorito en la superficie** hacen que el modelo apueste claramente por el favorito.

---

### Conclusiones generales

- **SHAP global** confirma que el modelo se basa principalmente en **win rates por superficie** y **ranking**.  
- **SHAP local** permite entender por qué en partidos concretos el modelo favorece o no al favorito.  
- En el caso **LOW**, el no favorito domina por rendimiento en superficie y ranking.  
- En el caso **HIGH**, el favorito domina gracias a la diferencia de ranking y la falta de experiencia del rival en la superficie.  

Esto demuestra que el modelo toma decisiones coherentes y basadas en información relevante.
